
# Fair Scheduler Story — Jobs View (Heaviest vs Medium vs Light Eval)

This notebook **tells a story** using **jobs** (not teams):
- **Heaviest job** (big model plan)
- **Medium job**
- **Light model evaluation job**

We compare a naive **BEFORE** scheduler (round‑robin) with an **AFTER** scheduler using **DRF (Dominant Resource Fairness)**.

You’ll get:
- **Step‑by‑step** allocation logs for BEFORE and AFTER, with a **reason** for each step.
- **Final allocations** for each job.
- **Visuals:** final dominant shares and how they evolve as we allocate more executors.



## Notes
- We model a single pool (e.g., **Batch**) with fixed **CPU** and **Memory** capacity.
- Each **executor** consumes a fixed amount of CPU & Memory.
- Jobs start with some allocations already (representing work already in progress).
- We then distribute the **next N executors**.
- **BEFORE = Round‑robin:** rotates blindly, ignores real usage.

- **AFTER = DRF:** gives the next executor to the job with the **lowest dominant share** (its bigger share of CPU or Mem).


In [ ]:

from dataclasses import dataclass, field
from typing import List, Dict, Tuple
import pandas as pd
import matplotlib.pyplot as plt

@dataclass
class Pool:
    name: str
    total_cpu: float
    total_mem_gb: float
    exec_cpu: float
    exec_mem_gb: float

@dataclass
class JobState:
    name: str
    cpu: float
    mem_gb: float
    max_executors: int
    executors: int = field(default=0)

    def can_receive(self) -> bool:
        return self.executors < self.max_executors

    def allocate_one(self, pool: Pool):
        self.cpu += pool.exec_cpu
        self.mem_gb += pool.exec_mem_gb
        self.executors += 1

def dominant_share(job: JobState, pool: Pool) -> float:
    cpu_share = job.cpu / pool.total_cpu if pool.total_cpu else 0.0
    mem_share = job.mem_gb / pool.total_mem_gb if pool.total_mem_gb else 0.0
    return max(cpu_share, mem_share)

def shares_snapshot(jobs: List[JobState], pool: Pool) -> Dict[str, Dict[str, float]]:
    snap = {}
    for j in jobs:
        cpu_share = j.cpu / pool.total_cpu if pool.total_cpu else 0.0
        mem_share = j.mem_gb / pool.total_mem_gb if pool.total_mem_gb else 0.0
        snap[j.name] = {
            "cpu_share": cpu_share,
            "mem_share": mem_share,
            "dominant_share": max(cpu_share, mem_share)
        }
    return snap

def round_robin_allocate(pool: Pool, jobs: List[JobState], extra_executors: int):
    history = []
    idx = 0
    steps = 0
    while steps < extra_executors:
        j = jobs[idx % len(jobs)]
        idx += 1
        if not j.can_receive():
            continue
        pre = shares_snapshot(jobs, pool)
        pre_dom = pre[j.name]['dominant_share']
        j.allocate_one(pool)
        post_dom = dominant_share(j, pool)
        steps += 1
        history.append({
            "step": steps,
            "allocated_to": j.name,
            "reason": "round-robin (fixed order, ignores real usage)",
            "dominant_share_before": round(pre_dom, 4),
            "dominant_share_after": round(post_dom, 4),
            "total_execs_for_job": j.executors
        })
    return jobs, history

def drf_allocate(pool: Pool, jobs: List[JobState], extra_executors: int):
    history = []
    for step in range(1, extra_executors+1):
        eligible = [j for j in jobs if j.can_receive()]
        if not eligible: break
        snaps = shares_snapshot(jobs, pool)
        eligible.sort(key=lambda j: (round(snaps[j.name]['dominant_share'], 12), j.name))
        chosen = eligible[0]
        pre_dom = snaps[chosen.name]['dominant_share']
        shares_reason = ", ".join([f"{n}: {snaps[n]['dominant_share']:.3f}" for n in sorted(snaps.keys())])
        chosen.allocate_one(pool)
        post_dom = dominant_share(chosen, pool)
        history.append({
            "step": step,
            "allocated_to": chosen.name,
            "reason": f"lowest dominant share wins → [{shares_reason}]",
            "dominant_share_before": round(pre_dom, 4),
            "dominant_share_after": round(post_dom, 4),
            "total_execs_for_job": chosen.executors
        })
    return jobs, history

def summarize(jobs: List[JobState], pool: Pool) -> pd.DataFrame:
    rows = []
    for j in jobs:
        cpu_share = j.cpu / pool.total_cpu if pool.total_cpu else 0.0
        mem_share = j.mem_gb / pool.total_mem_gb if pool.total_mem_gb else 0.0
        rows.append({
            "job": j.name,
            "cpu_alloc": j.cpu,
            "mem_alloc_gb": j.mem_gb,
            "cpu_share": round(cpu_share, 4),
            "mem_share": round(mem_share, 4),
            "dominant_share": round(max(cpu_share, mem_share), 4),
            "executors_added": j.executors
        })
    return pd.DataFrame(rows).sort_values(by=["dominant_share","job"]).reset_index(drop=True)

def clone_jobs(jobs: List[JobState]) -> List[JobState]:
    return [JobState(j.name, j.cpu, j.mem_gb, j.max_executors, j.executors) for j in jobs]



## 1) Scenario (you can tweak these numbers)
- **Pool:** 200 vCPU, 800 GB RAM. Each executor uses **2 vCPU** and **16 GB RAM**.
- **Jobs:**  
  - **Heaviest_Job** — big model plan (starts high on CPU and Mem: 60 vCPU, 240 GB).  
  - **Medium_Job** — medium feature/eval run (40 vCPU, 240 GB).  
  - **Light_Eval_Job** — quick model evaluation (20 vCPU, 40 GB).  
- **Ceilings:** max 30 new executors for any job (demo).  
- **We will distribute:** next **15 executors**.


In [ ]:

pool = Pool(name="Batch", total_cpu=200, total_mem_gb=800, exec_cpu=2, exec_mem_gb=16)

jobs_seed = [
    JobState(name="Heaviest_Job", cpu=60, mem_gb=240, max_executors=30),
    JobState(name="Medium_Job",   cpu=40, mem_gb=240, max_executors=30),
    JobState(name="Light_Eval_Job", cpu=20, mem_gb=40, max_executors=30),
]

extra_executors = 15



## 2) Run BEFORE (Round‑Robin) and AFTER (DRF)
We clone the same starting state, run each scheduler, and capture a detailed **step-by-step** log with **reasons**.


In [ ]:

jobs_rr = clone_jobs(jobs_seed)
jobs_drf = clone_jobs(jobs_seed)

jobs_rr_after, rr_history = round_robin_allocate(pool, jobs_rr, extra_executors=extra_executors)
jobs_drf_after, drf_history = drf_allocate(pool, jobs_drf, extra_executors=extra_executors)

rr_df = pd.DataFrame(rr_history)
drf_df = pd.DataFrame(drf_history)

print("=== BEFORE: Round‑Robin step log ===")
display(rr_df)

print("\n=== AFTER: DRF step log ===")
display(drf_df)



## 3) Final Allocations (Before vs After)


In [ ]:

sum_rr = summarize(jobs_rr_after, pool)
sum_drf = summarize(jobs_drf_after, pool)

print("=== BEFORE: Round‑Robin final allocation ===")
display(sum_rr)

print("\n=== AFTER: DRF final allocation ===")
display(sum_drf)

diff = sum_drf.set_index("job")[["cpu_alloc","mem_alloc_gb","dominant_share"]].subtract(
       sum_rr.set_index("job")[["cpu_alloc","mem_alloc_gb","dominant_share"]], fill_value=0).reset_index()

print("\n=== Difference (AFTER - BEFORE) ===")
display(diff)



## 4) Visual — Final Dominant Share (BEFORE: Round‑Robin)


In [ ]:

plt.figure(figsize=(6,4))
plt.bar(sum_rr["job"], sum_rr["dominant_share"])
plt.title("Final Dominant Share — BEFORE (Round‑Robin)")
plt.xlabel("Job")
plt.ylabel("Dominant Share (0.0–1.0)")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()



## 5) Visual — Final Dominant Share (AFTER: DRF)


In [ ]:

plt.figure(figsize=(6,4))
plt.bar(sum_drf["job"], sum_drf["dominant_share"])
plt.title("Final Dominant Share — AFTER (DRF)")
plt.xlabel("Job")
plt.ylabel("Dominant Share (0.0–1.0)")
plt.xticks(rotation=20)
plt.tight_layout()
plt.show()



## 6) Visual — DRF: Dominant Share Over Allocation Steps


In [ ]:

def drf_timeline(pool: Pool, jobs_seed: List[JobState], extra: int):
    jobs = clone_jobs(jobs_seed)
    timeline = {"step":[0]}
    snap0 = shares_snapshot(jobs, pool)
    for j in jobs:
        timeline[j.name] = [snap0[j.name]["dominant_share"]]
    for step in range(1, extra+1):
        eligible = [j for j in jobs if j.can_receive()]
        if not eligible: break
        snaps = shares_snapshot(jobs, pool)
        eligible.sort(key=lambda j: (round(snaps[j.name]['dominant_share'], 12), j.name))
        chosen = eligible[0]
        chosen.allocate_one(pool)
        timeline["step"].append(step)
        snaps2 = shares_snapshot(jobs, pool)
        for j in jobs:
            timeline[j.name].append(snaps2[j.name]["dominant_share"])
    return pd.DataFrame(timeline)

timeline_drf = drf_timeline(pool, jobs_seed, extra_executors)
plt.figure(figsize=(6,4))
for col in [c for c in timeline_drf.columns if c != "step"]:
    plt.plot(timeline_drf["step"], timeline_drf[col], label=col)
plt.title("AFTER (DRF) — Dominant Share Over Steps")
plt.xlabel("Allocation Step")
plt.ylabel("Dominant Share (0.0–1.0)")
plt.legend()
plt.tight_layout()
plt.show()



## 7) Visual — Round‑Robin: Dominant Share Over Allocation Steps


In [ ]:

def rr_timeline(pool: Pool, jobs_seed: List[JobState], extra: int):
    jobs = clone_jobs(jobs_seed)
    timeline = {"step":[0]}
    snap0 = shares_snapshot(jobs, pool)
    for j in jobs:
        timeline[j.name] = [snap0[j.name]["dominant_share"]]
    idx = 0
    steps = 0
    while steps < extra:
        j = jobs[idx % len(jobs)]
        idx += 1
        if not j.can_receive():
            continue
        j.allocate_one(pool)
        steps += 1
        timeline["step"].append(steps)
        snap = shares_snapshot(jobs, pool)
        for job in jobs:
            timeline[job.name].append(snap[job.name]["dominant_share"])
    return pd.DataFrame(timeline)

timeline_rr = rr_timeline(pool, jobs_seed, extra_executors)
plt.figure(figsize=(6,4))
for col in [c for c in timeline_rr.columns if c != "step"]:
    plt.plot(timeline_rr["step"], timeline_rr[col], label=col)
plt.title("BEFORE (Round‑Robin) — Dominant Share Over Steps")
plt.xlabel("Allocation Step")
plt.ylabel("Dominant Share (0.0–1.0)")
plt.legend()
plt.tight_layout()
plt.show()



## 8) Story Wrap‑Up (What to say)
- **Before:** Round‑robin blindly rotates, so the **Heaviest Job** might keep getting capacity even when the **Light Eval Job** is far behind. Unfair & unpredictable.

- **After:** DRF always allocates the next executor to the job with the **lowest dominant share** (furthest behind). Shares converge → **no job hogs the pool**.

- **Why each step happens:** The DRF step log shows each job’s share before allocation; the **lowest number wins** that step.

- **In production:** We’d also enforce **executor ceilings** per job and **per‑user concurrency caps** to prevent flooding, plus pool isolation for Interactive vs Batch.
